# 프로젝트:얼굴 인식하여 캐릭터 씌우기

## Face Detection vs Face Recognition

패키지 설치
> pip install mediapipe

In [34]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image):
    alpha = overlay_image[:, :, 3]#BGRA
    mask_image = alpha / 255 #1은 불투명 0은 투명
    
    for c in range(0,3):#BGR
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:,:,c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))
    

mp_face_detection = mp.solutions.face_detection#얼굴 검출
mp_drawing = mp.solutions.drawing_utils#얼굴 특징 그리기

#동영상 파일 열기 
cap = cv2.VideoCapture('face_video.mp4')

#이미지 불러오기
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED)#100 x 100
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED)#100 x 100
image_nose_tip = cv2.imread('nose_tip.png', cv2.IMREAD_UNCHANGED)#300 x 100


#with mp_face_detection.FaceDetection(얼굴인식 거리(0이면 5미터 이하 1이면 5미터 이상), 얼굴 인식 확률) as face_detection:
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")            
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#얼굴인식 모듈에서 RGB를 사용해서 변환해주기
        results = face_detection.process(image)#찾은 얼굴 

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)#원래댜로 되돌리기
        
        if results.detections:
            #6개의 특징(오른쪽눈, 왼쪽눈, 코끝부분, 입중심, 오른쪽 귀, 왼쪽귀(귀구멍 앞))
            for detection in results.detections:
                #mp_drawing.draw_detection(image, detection)#찾은 얼굴에 사각형 점 그리기
                #print(detection)
                
                #특정위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]#오른족 눈
                left_eye = keypoints[1]#왼족 눈
                nose_tip = keypoints[2]#코끝
                
                
                h, w, _ = image.shape#이미지 정보 얻어오기
                right_eye = (int(right_eye.x * w) - 20,int(right_eye.y * h) - 100)
                left_eye = (int(left_eye.x * w) + 20,int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w),int(nose_tip.y * h))
                
                #양눈에 원 그리기
                #cv2.circle(image, right_eye, 50,(255,0,0), 10, cv2.LINE_AA)
                #cv2.circle(image, left_eye, 50,(0,255,0), 10, cv2.LINE_AA)
                #cv2.circle(image, nose_tip, 75,(0,255,255), 10, cv2.LINE_AA)
                
                #눈과 코에 이미지 그려주기
                #이미지의 크기가 100이기때문에 주앙을 맞추기 위해 + - 50을 해줌
                #image[right_eye[1] - 50:right_eye[1] + 50, right_eye[0] - 50:right_eye[0] + 50] = image_right_eye
                #image[left_eye[1] - 50:left_eye[1] + 50, left_eye[0] - 50:left_eye[0] + 50] = image_left_eye
                #image[nose_tip[1] - 50:nose_tip[1] + 50, nose_tip[0] - 150:nose_tip[0] + 150] = image_nose_tip
                
                #image, x, y, w, h, overay_iamge
                overlay(image, *right_eye, 50,50,image_right_eye)
                overlay(image, *left_eye, 50,50,image_left_eye)
                overlay(image, *nose_tip, 150,50,image_nose_tip)
                
                
        # Flip the image horizontally for a selfie-view display.
        #cv2.imshow('MediaPipe Face Detection', cv2.flip(image, 1))
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None,fx=0.5,fy=0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

Ignoring empty camera frame.


In [28]:
img = cv2.imread('right_eye.png',cv2.IMREAD_UNCHANGED)
img.shape

(100, 100, 4)